# documentation
https://hex.tech/blog/connecting-snowflake-python/

In [1]:
from platform import python_version
print(python_version())

3.9.13


In [3]:
# packages
import pandas as pd
import os
import snowflake.connector

In [ ]:
%env snowflakeuser=<your_snowflake_username>
%env snowflakepass=<your_snowflake_password>

In [6]:
# snowflake connection
ctx = snowflake.connector.connect(
    user=os.environ['snowflakeuser'],
    password=os.environ['snowflakepass'],
    account='zib52348.us-east-1',
    warehouse='ANALYTICS',
    role='ACCOUNTADMIN',
    database='ANALYTICS',
    schema ='FORECASTING'
    )
cs = ctx.cursor()

In [7]:
sql = "select 1"
cs.execute(sql)
first_row = cs.fetchone()
print(first_row[0])

1
